# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 20901992


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/32 [00:00<?, ?it/s]

Rendering models:   6%|▋         | 2/32 [00:05<01:15,  2.51s/it]

Rendering models:  12%|█▎        | 4/32 [00:06<00:56,  2.03s/it]

Rendering models:  16%|█▌        | 5/32 [00:07<00:40,  1.52s/it]

Rendering models:  19%|█▉        | 6/32 [00:07<00:28,  1.10s/it]

...

Rendering models:  84%|████████▍ | 27/32 [00:12<00:01,  3.28it/s]

Rendering models:  91%|█████████ | 29/32 [00:12<00:00,  3.83it/s]

Rendering models:  94%|█████████▍| 30/32 [00:12<00:00,  3.91it/s]

Rendering models:  97%|█████████▋| 31/32 [00:13<00:00,  3.88it/s]

equidad1                              0.066616
not-logged-in-8c0c44faf754d344b18a    0.002139
not-logged-in-8c0c44faf754d344b18a    0.000163
Lechito73                             0.000465
not-logged-in-9c310346c8df6cdafc65    0.000792
alja314                               0.002823
klmasters                             0.000133
amerkg11                              0.000806
not-logged-in-5cb1e7dd7b917bacaab0    0.000431
not-logged-in-1991aa882f6b89978314    0.008364
bronzesword                           0.000369
Jessica_Korkmaz                       0.007368
not-logged-in-1c93afd4df6b0118a43a    0.032984
not-logged-in-a4a0edec2005410174b2    0.003411
databanana                            0.000194
TRISTANM18                            0.000100
peakscience7                          0.000108
RaulE                                 0.000250
merkosh                               0.000304
acapirala                             0.000110
not-logged-in-3ae935e3afaee843f530    0.000962
Lavadude     

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())